In [296]:
import math
import random
import numpy as np





class trash:

    def __init__(self,r_val) -> None:
        
        self.discard_deck = []
        self.draw_deck = []
        self.player1 = {'deck': [], 'visibility': [], 'array_count': [x for x in range(1,11)]}
        self.player2 = {'deck': [], 'visibility': [], 'array_count': [x for x in range(1,11)]}
        self.player1['visibility'] = ['X'] * self.player1['array_count'][-1]
        self.player2['visibility'] = ['X'] * self.player2['array_count'][-1]



        self.num_turns = 0
        self.transitions = 0
        self.winner = []
        self.game_over = False
        self.deck = self.build_deck()
        self.deck = self.shuffle_cards(self.deck)
        self.initial_set_decks()

    def build_deck(self):

        deck = []
        num_cards = 4
        c = 0
        for _ in (range(13)):
            #print(c)
            #add 4 of every card to deck
            deck.extend([c+1] * num_cards)

            c = c+1
        return deck

    def shuffle_cards(self,deck:list):

        #Fisher-Yates for shuffling

       
        n = len(deck)

        for c in range(n):
            r = random.uniform(.01,.999) #works for now
            p = int((r*(n-c))+c)
            #print(p)
            #print(c)

            #constant time complexity for this operation
            deck[c], deck[p] = deck[p], deck[c]

            #c = c+1
        return deck 

    #SETTRS

    def initial_set_decks(self):
        self.player1['deck'].extend(self.deck[10:20])
        self.player2['deck'].extend(self.deck[0:10])
        
        self.draw_deck.extend(self.deck[20:51]) # Update this line
        self.discard_deck.append(self.deck[-1])
        #print(len(self.player2['deck']) + len(self.player1['deck']) + len(self.draw_deck) + len(self.discard_deck))

    
    def can_place_card(self, card, player):
        #print('last_call can_place_card!')
        if card == 11:
            if 'X' in player['visibility']:
                return True
        elif card in [12, 13] or card > len(player['deck']):
            return False
        elif player['visibility'][card-1] == 'X':
            return True
        else:
            return False

    def place_card(self, card, player):
        #print('last_call place_card!')
        if card == 11:
            index = player['visibility'].index('X')
        else:
            index = card - 1
        old_card = player['deck'][index]
        player['deck'][index] = card
        player['visibility'][index] = card
        return old_card

    def is_game_over(self):
        #print('last_call is_game_over!')
        # The game is over when either player's array_count is empty.
        if len(self.player1['array_count']) == 0 or len(self.player2['array_count']) == 0:
            return True
        else: return False

    def is_array_cleared(self, player):
        """Checks if the player's array is cleared."""
        #print('Visibility Array:', player['visibility'])
        if 'X' in player['visibility']:
            #print('Array Not Cleared.')
            return False
        else:
            #print('Array Cleared!')
            return True


    def shuffle_and_reset_deck(self, current_player):
        #print('last_call shuffle_and_reset_deck!')
        shuffled_deck = current_player['deck'] + self.draw_deck + self.discard_deck
        shuffled_deck = self.shuffle_cards(shuffled_deck)
        #print(len(shuffled_deck))

        array_size = current_player['array_count'][-1]
        current_player['deck'] = shuffled_deck[:array_size]
        self.draw_deck = shuffled_deck[array_size:]
        self.discard_deck = []

        current_player['visibility'] = ['X'] * array_size

        

    def play_turn(self, player, opponent):

        initial_leader = "player" if len(player['array_count']) < len(opponent['array_count']) else "opponent"


        if len(self.draw_deck) == 0:
            self.draw_deck = self.shuffle_cards(self.discard_deck[:-1])
            self.discard_deck = [self.discard_deck[-1]]
        
        if self.can_place_card(self.discard_deck[-1],player):
            card_in_hand = self.discard_deck.pop()
        else:
            card_in_hand = self.draw_deck.pop()


        #print('card in hand', card_in_hand)
        #print('visibility', player['visibility'])
        #print('deck', player['deck'])
        while self.can_place_card(card_in_hand, player):

            #print('in loop!')
            card_in_hand = self.place_card(card_in_hand, player)

            if card_in_hand in [12, 13]:  # Queen and King are represented by 12 and 13
                break

            #print('card in hand loop', card_in_hand)    
            if self.is_array_cleared(player):
                player['array_count'].pop()
                
                if not self.is_game_over():
                    self.shuffle_and_reset_deck(player)
        
        
        if self.is_array_cleared(player):
            
            if player['array_count']: player['array_count'].pop()
            
            if not self.is_game_over():
                self.shuffle_and_reset_deck(player)
            
        
        self.discard_deck.append(card_in_hand)

        

        current_leader = "player" if len(player['array_count']) < len(opponent['array_count']) else "opponent"
        if initial_leader != current_leader:
            self.transitions += 1






    def play_game(self):
        while not self.is_game_over():
            #print("player1")
            self.play_turn(self.player1, self.player2)
            self.num_turns += 1

            if self.is_game_over():  # check if the game ended after player1's turn
                break
            #print("player2")
            self.play_turn(self.player2, self.player1)
            self.num_turns += 1
                # Determine the winner
        if len(self.player1['array_count']) == 0:
            self.winner = "Player 1"
        else:
            self.winner = "Player 2"
    
    def play_game_check(self):
        for i in range(100):
            print("player1")
            self.play_turn(self.player1, self.player2)
            self.num_turns += 1

            if self.is_game_over():  # check if the game ended after player1's turn
                break
            print("player2")
            self.play_turn(self.player2, self.player1)
            self.num_turns += 1


            


        # Determine the winner
        if len(self.player1['array_count']) == 0:
            self.winner = "Player 1"
        else:
            self.winner = "Player 2"



In [297]:
trash_sim = trash(.2)

#Consistency check


game = trash(1)

game.play_game()
print(f"The winner is {game.winner} after {game.num_turns} turns with {game.transitions} transitions!")


The winner is Player 2 after 174 turns with 9 transitions!


In [298]:
def shuffle_and_reset_deck(self, current_player):
        print('discard len', len(self.discard_deck))
        print('draw len', len(self.draw_deck))
        print('current player len', len(current_player['deck']))
        # This function resets the game state by shuffling all cards and dealing new arrays.
        all_cards = current_player['deck'] + self.draw_deck + self.discard_deck
        #print(len(current_player['deck']))
        shuffled_cards = self.shuffle_cards(all_cards)
        
        print('len shuffle deck b4:', len(shuffled_cards))


        next_array_count = current_player['array_count'][-1] # Decrease the size of the array for the next round
        #print(next_array_count)
        current_player['deck'] = shuffled_cards[0: next_array_count]
        print('len currnt deck:',len(current_player['deck']))

        # Update the draw and discard piles
        self.draw_deck = shuffled_cards[next_array_count:]
        print('shuffle deck', len(shuffled_cards[next_array_count:]))